In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()


In [ ]:
def count_variants(chr, pos):
    if pos == 0:
        query = (
            "SELECT chr_id_b37, position_b37, COUNT(*) as count FROM thegwensproject.genomiX.variants WHERE chr_id_b37 = '" +str(chr) +"' GROUP BY chr_id_b37, position_b37 HAVING COUNT(*) > 1 LIMIT 20;")
    else:
        query = (
            "SELECT chr_id_b37, position_b37, COUNT(*) as count FROM thegwensproject.genomiX.variants WHERE chr_id_b37 = '" +str(chr) +"' AND position_b37 = " + str(pos) + " GROUP BY chr_id_b37, position_b37 HAVING COUNT(*) > 1 LIMIT 20;")
    return query
query_job = client.query(count_variants(6, 166956723))
for row in query_job:
    # Row values can be accessed by field name or index.
    print(row)

In [ ]:
def variant_info(chr, pos):
    query = (
        "SELECT * FROM thegwensproject.genomiX.variants WHERE chr_id_b37 = '" +str(chr) +"' AND position_b37 = " + str(pos) + " LIMIT 20;")
    return query

query_job = client.query(variant_info(6, 166956723))
for row in query_job:
    # Row values can be accessed by field name or index.
    print(row[9])
    print (row)

In [ ]:
def tag_disease_variant_info(chr, pos):
    query = (
        "SELECT * FROM thegwensproject.genomiX.disease_variant_gene WHERE tag_chrom = '" +str(chr) +"' AND tag_pos = "+ str(pos) +"  LIMIT 20;")
    return query

query_job = client.query(tag_disease_variant_info(6, 166952906))
for row in query_job:
    # Row values can be accessed by field name or index.
    print(row['lead_pos'])
    print (row)

In [ ]:
def lead_disease_variant_info(chr, pos):
    query = (
        "SELECT * FROM thegwensproject.genomiX.disease_variant_gene WHERE lead_chrom = '" +str(chr) +"' AND lead_pos = "+ str(pos) +"  LIMIT 20;")
    return query
query_job = client.query(lead_disease_variant_info(6, 166956723))
for row in query_job:
    # Row values can be accessed by field name or index.
    print (row)

In [ ]:
def lead_disease_info(chr, pos):
    query = (
        "SELECT * FROM thegwensproject.genomiX.variant_disease WHERE lead_chrom = '" +str(chr) +"' AND lead_pos = "+ str(pos) +"  LIMIT 20;")
    return query
query_job = client.query(lead_disease_info(6, 166956723))
for row in query_job:
    # Row values can be accessed by field name or index.
    print (row['trait_reported'])